In [ ]:
# Author: Renzo Caballero
# KAUST: King Abdullah University of Science and Technology
# email: renzo.caballerorosas@kaust.edu.sa caballerorenzo@hotmail.com
# Website: renzocaballero.org, https://github.com/RenzoCab
# September 2023; Last revision: 14/09/2023

from scipy.io import loadmat
import csv
import numpy as np
import matplotlib.pyplot as plt
import random

corr_x    = [];
corr_y    = [];
x_ideal_t = [];
x_corr_t  = [];
u_x       = [];
y_ideal_t = [];
y_corr_t  = [];
u_y       = [];
num_lay   = [];
star_x    = [];
star_y    = [];
failing_x = [];
failing_y = [];
more_prob = [];
block_x   = [];
block_y   = [];
u_x_block = [];
u_y_block = [];

# Load the MATLAB file
data = loadmat('add_dudw_data.mat');
with open('100506_1.csv', 'r') as csv_file:
    csv_reader = csv.reader(csv_file)
    for row in csv_reader:
        corr_x.append(int(row[1]))
        corr_y.append(int(row[2]))
        num_lay.append(int(row[3]))
        x_ideal_t.append(int(row[4]))
        x_corr_t.append(float(row[5]))
        u_x.append(int(row[6]))
        y_ideal_t.append(int(row[9]))
        y_corr_t.append(float(row[10]))
        u_y.append(int(row[11]))
        star_x.append(int(row[14]))
        star_y.append(int(row[15]))
        failing_x.append(float(row[16]))
        failing_y.append(float(row[17]))
        more_prob.append(float(row[18]))
        block_x.append(int(row[19]))
        block_y.append(int(row[20]))
        u_x_block.append(int(row[21]))
        u_y_block.append(int(row[22]))
        
# Access MATLAB variables as Python dictionaries
array_corruptions = data['all_data']
array_corruptions = array_corruptions[0:5000];
print('The number of corruptions is',len(array_corruptions))
print('The length of each corruption is',len(array_corruptions[0]))

# x should be between 0 and 1100 (here we just choose 500 as an example):
x_ideal = 500;
x_corr  = 500;
# c (corruption) should be between 0 and 4999 (here we just choose 600 as an example):
c = array_corruptions[600];
# u (the control) should be either 0, 1, or -1 (here we just choose 1):
u = 1;
x_ideal_next = x_ideal + u;
x_corr_next  = x_corr  + u*c[x_ideal];
print('Then we move ideally from',x_ideal,'to',x_ideal_next)
print('Then we move corrupted from',x_corr,'to',x_corr_next)

In [ ]:
num_layers = len(np.unique(num_lay))
elem_layer = np.unique(num_lay)

corruption_x = array_corruptions[int(corr_x[0])-1];
corruption_y = array_corruptions[int(corr_y[0])-1];

inc = 0;
accum_x = 0;
accum_y = 0;

#star_x.append(int(row[14]))
#star_y.append(int(row[15]))
#failing_x.append(float(row[16]))
#failing_y.append(float(row[17]))
#more_prob.append(float(row[18]))
#block_x.append(int(row[19]))
#block_y.append(int(row[20]))
#u_x_block.append(int(row[21]))
#u_y_block.append(int(row[22]))

star_x    = star_x[0];
star_y    = star_y[0];
failing_x = failing_x[0];
failing_y = failing_y[0];
more_prob = more_prob[0];

u_x_ideal = u_x;
u_y_ideal = u_y;

for i in range(num_layers):
    
    for j in range(len(num_lay)):
        index = num_lay.index(elem_layer[i])

    we_start_in = index
    simulated_x_ideal = [x_ideal_t[we_start_in]];
    simulated_x_corr  = [x_corr_t[we_start_in]];
    simulated_y_ideal = [y_ideal_t[we_start_in]];
    simulated_y_corr  = [y_corr_t[we_start_in]];
    
    k = we_start_in;
    count_x = 0;
    count_y = 0;
    
    for j in range(num_lay.count(elem_layer[i])):
        
        if u_x[k+j] == 1 and count_x < star_x:
            count_x = count_x + 1;
            u_x[k+j] = 0;
        elif u_x[k+j] == 1 and count_x >= star_x:
            aux = 1;
        elif u_x[k+j] == 0:
            count_x = 0;
            
        if u_y[k+j] == 1 and count_y < star_y:
            count_y = count_y + 1;
            u_y[k+j] = 0;
        elif u_y[k+j] == 1 and count_y >= star_y:
            aux = 1;
        elif u_y[k+j] == 0:
            count_y = 0;
        
        if j > 1:
            
            if u_y[j+k-1] == 1 and u_y[j+k] == 1:
                blocked_x = random.random() < failing_x * more_prob;
            else:
                blocked_x = random.random() < failing_x;

            if u_x[j+k-1] == 1 and u_x[j+k] == 1:
                blocked_y = random.random() < failing_y * more_prob;
            else:
                blocked_y = random.random() < failing_y;
            
            if blocked_x == 1:
                u_x[k+j] = 0;
            if blocked_y == 1:
                u_y[k+j] = 0;
        
        simulated_x_ideal.append(simulated_x_ideal[j]+u_x_ideal[k+j])
        simulated_x_corr.append(simulated_x_corr[j]+u_x[k+j]);
        simulated_y_ideal.append(simulated_y_ideal[j]+u_y_ideal[k+j])
        simulated_y_corr.append(simulated_y_corr[j]+u_y[k+j]);
        
        accum_x = accum_x + abs(simulated_x_corr[j]-x_corr_t[k+j]);
        accum_y = accum_y + abs(simulated_y_corr[j]-y_corr_t[k+j]);
        inc = inc + 1;
        print('X_',inc,'_',accum_x);
        print('Y_',inc,'_',accum_y);
        #print(u_x[k+j],' _',u_x_block[k+j]);
        #print(u_y[k+j],' _',u_y_block[k+j]);


In [ ]:
#plt.plot(simulated_x_ideal[0:9000], simulated_y_ideal[0:9000])
#plt.plot(simulated_x_corr[0:9000], simulated_y_corr[0:9000])
#plt.plot(x_corr_t[0:9000], y_corr_t[0:9000])